<a href="https://colab.research.google.com/github/korayaslaan/artificial_intelligence/blob/main/Sputniknews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import os
import bs4
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
import time
import urllib.request
import re
import urllib3
from pandas import DataFrame
import csv
import datetime
from datetime import datetime, timedelta
import os
import csv
import concurrent
import multiprocessing
from multiprocessing import pool
import io
from pprint import pprint

In [56]:
def dateCreator():
  science = []
  tech = []
  #https://sputniknews.com/services/search/getmore/?query=science&tags_limit=20&tags=0&interval=period&date_from=2021-12-01&date_to=2021-12-01   2005-02-01
  ser_date = pd.Series(pd.date_range('20050201', periods=6180))
  linkscience = "https://sputniknews.com/services/search/getmore/?query=science&tags_limit=20&tags=0&interval=period&date_from="
  linktech = "https://sputniknews.com/services/search/getmore/?query=tech&tags_limit=20&tags=0&interval=period&date_from="
  link2 = "&date_to="
  a = 6150
  b = 6179  #6179 son 0 dan başlicak
  for j in range(a,b):
    dateEnd  = ser_date[j].strftime("%Y-%m-%d")
    science.append("{}{}{}{}".format(linkscience,dateEnd,link2,dateEnd))

  for j in range(a,b):#6179 son 0 dan başlicak
    dateEnd  = ser_date[j].strftime("%Y-%m-%d")
    tech.append("{}{}{}{}".format(linktech,dateEnd,link2,dateEnd))
    
  return science,tech 

In [57]:
dateCreator()

(['https://sputniknews.com/services/search/getmore/?query=science&tags_limit=20&tags=0&interval=period&date_from=2021-12-04&date_to=2021-12-04',
  'https://sputniknews.com/services/search/getmore/?query=science&tags_limit=20&tags=0&interval=period&date_from=2021-12-05&date_to=2021-12-05',
  'https://sputniknews.com/services/search/getmore/?query=science&tags_limit=20&tags=0&interval=period&date_from=2021-12-06&date_to=2021-12-06',
  'https://sputniknews.com/services/search/getmore/?query=science&tags_limit=20&tags=0&interval=period&date_from=2021-12-07&date_to=2021-12-07',
  'https://sputniknews.com/services/search/getmore/?query=science&tags_limit=20&tags=0&interval=period&date_from=2021-12-08&date_to=2021-12-08',
  'https://sputniknews.com/services/search/getmore/?query=science&tags_limit=20&tags=0&interval=period&date_from=2021-12-09&date_to=2021-12-09',
  'https://sputniknews.com/services/search/getmore/?query=science&tags_limit=20&tags=0&interval=period&date_from=2021-12-10&date_t

In [58]:
def get_link_tech(i):
     count = 0
     r = requests.get(i)   
     soup = BeautifulSoup(r.content, 'html5lib')
     for a in soup.find_all('a', href=True):
       count+=1
       link = a['href']
       result = link.endswith("ml")
       if(result == True):
         if(count % 2 == 0):
           url  =   "https://sputniknews.com/"
           linkson = "{}{}".format(url,link)
           with open("TechLink.txt", 'a') as file: 
               file.write(linkson+'\n')

In [59]:
def get_link_science(i):
     count = 0
     r = requests.get(i)   
     soup = BeautifulSoup(r.content, 'html5lib')
     for a in soup.find_all('a', href=True):
       count+=1
       link = a['href']
       result = link.endswith("ml")
       if(result == True):
         if(count % 2 == 0):
           url  =   "https://sputniknews.com/"
           linkson = "{}{}".format(url,link)
           with open("ScienceLink.txt", 'a') as file: 
               file.write(linkson+'\n')

In [60]:
def creator_science(url):
    r = requests.get(url)   
    soup = BeautifulSoup(r.content, 'html5lib')
    title = soup.find("h1").getText()
    content_array = soup.find("div", attrs = {"class":"article__body"}).getText()
    # category = soup.find("a", attrs = {"class":"dcr-yx39j8"}).getText()
    category = "science"
    date = soup.find("a", attrs = {"class":"convert-date"}).getText()
    date = date.split()
    date =  date[2]
    content_array = content_array.split()
    content_string = ""
    for w in content_array:
       content_string = content_string+" "+w
    w_data="{};{};{};{};{}".format(url,date,category,title,content_string)
    with open("Creator.txt", 'a') as file: 
        file.write(w_data+'\n')

In [61]:
def creator_tech(url):
    r = requests.get(url)   
    soup = BeautifulSoup(r.content, 'html5lib')
    title = soup.find("h1").getText()
    content_array = soup.find("div", attrs = {"class":"article__body"}).getText()
    # category = soup.find("a", attrs = {"class":"dcr-yx39j8"}).getText()
    category = "tech"
    date = soup.find("a", attrs = {"class":"convert-date"}).getText()
    date = date.split()
    date =  date[2]
    content_array = content_array.split()
    content_string = ""
    for w in content_array:
       content_string = content_string+" "+w
    w_data="{};{};{};{};{}".format(url,date,category,title,content_string)
    with open("Creator.txt", 'a') as file: 
        file.write(w_data+'\n')

In [62]:
science,tech= dateCreator()

with concurrent.futures.ProcessPoolExecutor() as execut:
  b_res=[execut.submit(get_link_science,i.strip()) for i in science]

with concurrent.futures.ProcessPoolExecutor() as execut:
  b_res=[execut.submit(get_link_tech,i.strip()) for i in tech]

In [63]:
SputnikNewsScience=[]
with open("ScienceLink.txt",'r',newline='') as f:
  for i in f.readlines():    
    SputnikNewsScience.append(i)

t1=time.time()
with concurrent.futures.ProcessPoolExecutor() as execut:
  b_res=[execut.submit(creator_science,i.strip()) for i in SputnikNewsScience]

print(time.time()-t1)

63.62505626678467


In [64]:
SputnikNewsTech=[]
with open("TechLink.txt",'r',newline='') as f:
  for i in f.readlines():    
    SputnikNewsTech.append(i)

t1=time.time()
with concurrent.futures.ProcessPoolExecutor() as execut:
  b_res=[execut.submit(creator_tech,i.strip()) for i in SputnikNewsTech]

print(time.time()-t1)

82.20865845680237
